# 패키지 로드

In [51]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time
import requests
import csv

# 기본 함수 설정

상품코드 추출 함수

In [14]:
def goodsno_list(goodsno):
    goodsno = []
    num_goods = int(re.findall(r'\d+', driver.find_element(By.XPATH, '/html/body/div/div[3]/div/div/div/div[2]/div[1]/div').text)[0])
    # 상품 페이지 수 계산
    if num_goods % 12 == 0:
        goods_page = num_goods / 12
    else:
        goods_page = int(num_goods / 12) + 1
    # 상품코드 추출
    for page_no in range(1, goods_page+1):
        url_goodsno = f'https://www.kurly.com/search?sword={keyword}&page={page_no}&filters=category%{category}'
        driver.get(url_goodsno)
        time.sleep(1)
        if page_no == goods_page:
            for num in range(1, (num_goods%12)+1):
                goodsno_num = re.findall(r'\d+', driver.find_element(By.XPATH, f'/html/body/div/div[3]/div/div/div/div[2]/div[3]/a[{num}]').get_attribute('href'))[0]
                goodsno.append(goodsno_num)
        else:
            for num in range(1, 10):
                goodsno_num = re.findall(r'\d+', driver.find_element(By.XPATH, f'/html/body/div/div[3]/div/div/div/div[2]/div[3]/a[{num}]').get_attribute('href'))[0]
                goodsno.append(goodsno_num)
    return goodsno

리뷰 추출 함수

In [32]:
# 제품별 리뷰 추출
def review_list(good, review):
    url = f'https://www.kurly.com/goods/{good}'
    driver.get(url)
    time.sleep(4)
    # 리뷰 수 추출
    review_tot_temp = re.findall(r'\d+', driver.find_element(By.XPATH, '/html/body/div/div/div[3]/div[2]/nav/ul/li[3]/a/span[2]').text)
    review_tot = int(''.join(review_tot_temp))
    review_in_page = 10
    if review_tot%10 == 0:
        page_review = int(review_tot/10)
        lastpage_review = 10
    else:
        page_review = int(review_tot/10) + 1
        lastpage_review = review_tot%10
    # 크롤링
    for _ in range(page_review):
        if _ == (page_review-1):
            review_in_page = lastpage_review
        for n in range(4, 4+review_in_page):
            try:
                name = driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/div/div/span[3]').text
                grade = driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/div/div/span[2]').text
            except:
                name = driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/div/div/span[2]').text
                grade = driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/div/div/span[1]').text
            menu = driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/article/div/div/h3').text
            review_txt = driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/article/div/p').text
            review_date = driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/article/div/footer/div/span').text
            try:
                like = int(re.findall(r'\d+', driver.find_element(By.XPATH, f'/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[{n}]/article/div/footer/button/span[2]').text)[0])
            except:
                like = 0
            review.append([good, grade, name, menu, review_txt, review_date, like])
        if _ != (page_review-1):
            driver.find_element(By.XPATH, '/html/body/div/div/div[3]/div[2]/div/div[3]/section/div[2]/div[14]/button[2]').click()
            time.sleep(1)

# 리뷰 수집

키워드 및 카테고리 설정

In [13]:
keyword = '솥밥'
category = '3A911006'
path = r"C:/Users/NT550-048/Desktop/chromedriver.exe"
url_goodsno = f'https://www.kurly.com/search?sword={keyword}&page=1&filters=category%{category}'

상품코드 추출

In [44]:
# test
s = Service(path)
driver = webdriver.Chrome(service=s)
driver.get(url_goodsno)
time.sleep(2)
goodsno = goodsno_list(goodsno)

상품리뷰 추출

In [ ]:
review = []
for good in goodsno:
    review_list(good, review)

데이터프레임 저장

In [46]:
review_df = pd.DataFrame(review, columns = ['ProductCode', 'MemberGrade', 'MemberName', 'ProductName', 'ReviewText', 'ReviewDate', 'ReviewLike'])

csv파일로 저장

In [48]:
review_df.to_csv('C:/Users/NT550-048/Desktop/잇더/temp.csv')

In [ ]:
# csv 저장/로드 오류시 사용
import csv
review_df.to_csv('C:/Users/NT550-048/Desktop/잇더/temp.csv', doublequote=False, escapechar='"', quoting=csv.QUOTE_NONE)